In [65]:
import numpy as np
#import sys

In [66]:
#functions of non-linear activations
def f_sigmoid(X, deriv=False):
    if not deriv:
        return 1 / (1 + np.exp(-X))
    else:
        return f_sigmoid(X)*(1 - f_sigmoid(X))


def f_softmax(X):
    Z = np.sum(np.exp(X), axis=1)
    Z = Z.reshape(Z.shape[0], 1)
    return np.exp(X) / Z

def f_relu(X):
    return np.maximum(0, X)


In [67]:
def exit_with_err(err_str):
    print >> sys.stderr, err_str
    sys.exit(1)

In [68]:
#Functionality of a single hidden layer
class Layer:
    def __init__(self, size, batch_size, is_input=False, is_output=False,
                 activation=f_sigmoid):
        self.is_input = is_input
        self.is_output = is_output

        # Z is the matrix that holds output values
        self.Z = np.zeros((batch_size, size[0]))
        # The activation function is an externally defined function (with a
        # derivative) that is stored here
        self.activation = activation

        # W is the outgoing weight matrix for this layer
        self.W = None
        # S is the matrix that holds the inputs to this layer
        self.S = None
        # D is the matrix that holds the deltas for this layer
        self.D = None
        # Fp is the matrix that holds the derivatives of the activation function
        self.Fp = None

        if not is_input:
            self.S = np.zeros((batch_size, size[0]))
            self.D = np.zeros((batch_size, size[0]))

        if not is_output:
            self.W = np.random.normal(size=size, scale=1E-4)

        if not is_input and not is_output:
            self.Fp = np.zeros((size[0], batch_size))

    def forward_propagate(self):
        if self.is_input:
            return self.Z.dot(self.W)

        self.Z = self.activation(self.S)
        if self.is_output:
            return self.Z
        else:
            # For hidden layers, we add the bias values here
            self.Z = np.append(self.Z, np.ones((self.Z.shape[0], 1)), axis=1)
            self.Fp = self.activation(self.S, deriv=True).T
            return self.Z.dot(self.W)


In [69]:
class MultiLayerPerceptron:
    def __init__(self, layer_config, batch_size=100, activationOut=f_softmax):
        self.layers = []
        self.num_layers = len(layer_config)
        self.minibatch_size = batch_size

        for i in range(self.num_layers-1):
            if i == 0:
                print ("Initializing input layer with size {0}.".format(layer_config[i]))
                # Here, we add an additional unit at the input for the bias
                # weight.
                self.layers.append(Layer([layer_config[i]+1, layer_config[i+1]],
                                         batch_size,
                                         is_input=True))
            else:
                print ("Initializing hidden layer with size {0}.".format(layer_config[i]))
                # Here we add an additional unit in the hidden layers for the
                # bias weight.
                self.layers.append(Layer([layer_config[i]+1, layer_config[i+1]],
                                         batch_size,
                                         activation=f_sigmoid))

        print ("Initializing output layer with size {0}.".format(layer_config[-1]))
        self.layers.append(Layer([layer_config[-1], None],
                                 batch_size,
                                 is_output=True,
                                 activation=activationOut))
        print ("Done!")

    def forward_propagate(self, data):
        # We need to be sure to add bias values to the input
        self.layers[0].Z = np.append(data, np.ones((data.shape[0], 1)), axis=1)

        for i in range(self.num_layers-1):
            self.layers[i+1].S = self.layers[i].forward_propagate()
        return self.layers[-1].forward_propagate()

    def backpropagate(self, yhat, labels):
        
        #---------------QUESTION 1 -----------#
        #exit_with_err("FIND ME IN THE CODE, What is computed in the next line of code?\n")
        # The error/distance between the calculated output and the "True" values
        
        self.layers[-1].D = (yhat - labels).T
        for i in range(self.num_layers-2, 0, -1):
            # We do not calculate deltas for the bias values
            W_nobias = self.layers[i].W[0:-1, :]
            
            # ----------------QUESTION 2---------------------#
            #exit_with_err("FIND ME IN THE CODE, What does this 'for' loop do?\n")
            # Updates the delta for each layer through backpropagation
            
            self.layers[i].D = W_nobias.dot(self.layers[i+1].D) * self.layers[i].Fp

    def update_weights(self, eta):
        for i in range(0, self.num_layers-1):
            W_grad = -eta*(self.layers[i+1].D.dot(self.layers[i].Z)).T
            self.layers[i].W += W_grad

    def evaluate(self, train_data, train_labels, test_data, test_labels,
                 num_epochs=70, eta=0.05, eval_train=False, eval_test=True):

        N_train = len(train_labels)*len(train_labels[0])
        N_test = len(test_labels)*len(test_labels[0])

        print ("Training for {0} epochs...".format(num_epochs))
        for t in range(0, num_epochs):
            out_str = "[{0:4d}] ".format(t)

            for b_data, b_labels in zip(train_data, train_labels):
                output = self.forward_propagate(b_data)
                self.backpropagate(output, b_labels)
                
                #--------------QUESTION 3----------------#
                #exit_with_err("FIND ME IN THE CODE, How does weight update is implemented? What is eta?\n")
                # eta = How agressive the weight update should be
                # Dot product between the next delta and the curreent layer output, 
                # then multiply with -eta, this value is added to the current weight
                
                self.update_weights(eta=eta)

            if eval_train:
                errs = 0
                for b_data, b_labels in zip(train_data, train_labels):
                    output = self.forward_propagate(b_data)
                    yhat = np.argmax(output, axis=1)
                    errs += np.sum(1-b_labels[np.arange(len(b_labels)), yhat])

                out_str = ("{0} Training error: {1:.5f}".format(out_str,
                                                           float(errs)/N_train))

            if eval_test:
                errs = 0
                for b_data, b_labels in zip(test_data, test_labels):
                    output = self.forward_propagate(b_data)
                    yhat = np.argmax(output, axis=1)
                    errs += np.sum(1-b_labels[np.arange(len(b_labels)), yhat])

                out_str = ("{0} Test error: {1:.5f}").format(out_str,
                                                       float(errs)/N_test)

            print (out_str)


In [70]:
def label_to_bit_vector(labels, nbits):
    bit_vector = np.zeros((labels.shape[0], nbits))
    for i in range(labels.shape[0]):
        bit_vector[i, labels[i]] = 1.0

    return bit_vector

In [71]:
def create_batches(data, labels, batch_size, create_bit_vector=False):
    N = data.shape[0]
    print ("Batch size {0}, the number of examples {1}.".format(batch_size,N))

    if N % batch_size != 0:
        print ("Warning in create_minibatches(): Batch size {0} does not " \
              "evenly divide the number of examples {1}.".format(batch_size,N))
    chunked_data = []
    chunked_labels = []
    idx = 0
    while idx + batch_size <= N:
        chunked_data.append(data[idx:idx+batch_size, :])
        if not create_bit_vector:
            chunked_labels.append(labels[idx:idx+batch_size])
        else:
            bit_vector = label_to_bit_vector(labels[idx:idx+batch_size], 10)
            chunked_labels.append(bit_vector)

        idx += batch_size

    return chunked_data, chunked_labels


In [72]:
def prepare_for_backprop(batch_size, Train_images, Train_labels, Valid_images, Valid_labels):
    
    print ("Creating data...")
    batched_train_data, batched_train_labels = create_batches(Train_images, Train_labels,
                                              batch_size,
                                              create_bit_vector=True)
    batched_valid_data, batched_valid_labels = create_batches(Valid_images, Valid_labels,
                                              batch_size,
                                              create_bit_vector=True)
    print ("Done!")


    return batched_train_data, batched_train_labels,  batched_valid_data, batched_valid_labels



In [73]:
from keras.datasets import mnist

In [74]:
(Xtr, Ltr), (X_test, L_test)=mnist.load_data()

Xtr = Xtr.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
Xtr = Xtr.astype('float32')
X_test = X_test.astype('float32')
Xtr /= 255
X_test /= 255
print(Xtr.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


60000 train samples
10000 test samples


# eta = 0.05

In [75]:
batch_size=100;

train_data, train_labels, valid_data, valid_labels=prepare_for_backprop(batch_size, Xtr, Ltr, X_test, L_test)

mlp = MultiLayerPerceptron(layer_config=[784, 100, 100, 10], batch_size=batch_size)

mlp.evaluate(train_data, train_labels, valid_data, valid_labels,
             eval_train=True)

print("Done:)\n")


Creating data...
Batch size 100, the number of examples 60000.
Batch size 100, the number of examples 10000.
Done!
Initializing input layer with size 784.
Initializing hidden layer with size 100.
Initializing hidden layer with size 100.
Initializing output layer with size 10.
Done!
Training for 70 epochs...
[   0]  Training error: 0.58533 Test error: 0.58690
[   1]  Training error: 0.08298 Test error: 0.07900
[   2]  Training error: 0.06225 Test error: 0.06270
[   3]  Training error: 0.05202 Test error: 0.05480
[   4]  Training error: 0.03803 Test error: 0.04280
[   5]  Training error: 0.03225 Test error: 0.03870
[   6]  Training error: 0.02890 Test error: 0.03900
[   7]  Training error: 0.02407 Test error: 0.03560
[   8]  Training error: 0.02075 Test error: 0.03480
[   9]  Training error: 0.02567 Test error: 0.03920
[  10]  Training error: 0.01623 Test error: 0.03140
[  11]  Training error: 0.02127 Test error: 0.03520
[  12]  Training error: 0.02045 Test error: 0.03400
[  13]  Trainin

# eta = 0.005

In [26]:
batch_size=100;

train_data, train_labels, valid_data, valid_labels=prepare_for_backprop(batch_size, Xtr, Ltr, X_test, L_test)

mlp = MultiLayerPerceptron(layer_config=[784, 100, 100, 10], batch_size=batch_size)

mlp.evaluate(train_data, train_labels, valid_data, valid_labels,
             eval_train=True, eta=0.005)

print("Done:)\n")
    


Creating data...
Batch size 100, the number of examples 60000.
Batch size 100, the number of examples 10000.
Done!
Initializing input layer with size 784.
Initializing hidden layer with size 100.
Initializing hidden layer with size 100.
Initializing output layer with size 10.
Done!
Training for 70 epochs...
[   0]  Training error: 0.70337 Test error: 0.70090
[   1]  Training error: 0.64723 Test error: 0.64320
[   2]  Training error: 0.59970 Test error: 0.59920
[   3]  Training error: 0.45023 Test error: 0.46170
[   4]  Training error: 0.20532 Test error: 0.19630
[   5]  Training error: 0.11045 Test error: 0.10890
[   6]  Training error: 0.08892 Test error: 0.08490
[   7]  Training error: 0.07593 Test error: 0.07530
[   8]  Training error: 0.06508 Test error: 0.06530
[   9]  Training error: 0.05532 Test error: 0.05620
[  10]  Training error: 0.04832 Test error: 0.05020
[  11]  Training error: 0.04265 Test error: 0.04460
[  12]  Training error: 0.03848 Test error: 0.03990
[  13]  Trainin

# eta = 0.5

In [27]:
batch_size=100;

train_data, train_labels, valid_data, valid_labels=prepare_for_backprop(batch_size, Xtr, Ltr, X_test, L_test)

mlp = MultiLayerPerceptron(layer_config=[784, 100, 100, 10], batch_size=batch_size)

mlp.evaluate(train_data, train_labels, valid_data, valid_labels,
             eval_train=True, eta=0.5)

print("Done:)\n")

Creating data...
Batch size 100, the number of examples 60000.
Batch size 100, the number of examples 10000.
Done!
Initializing input layer with size 784.
Initializing hidden layer with size 100.
Initializing hidden layer with size 100.
Initializing output layer with size 10.
Done!
Training for 70 epochs...
[   0]  Training error: 0.90128 Test error: 0.90200
[   1]  Training error: 0.90128 Test error: 0.90200
[   2]  Training error: 0.90965 Test error: 0.91080
[   3]  Training error: 0.88763 Test error: 0.88650
[   4]  Training error: 0.90248 Test error: 0.90260
[   5]  Training error: 0.90263 Test error: 0.90180
[   6]  Training error: 0.89782 Test error: 0.89900
[   7]  Training error: 0.89782 Test error: 0.89900
[   8]  Training error: 0.90070 Test error: 0.89680
[   9]  Training error: 0.90248 Test error: 0.90260
[  10]  Training error: 0.89782 Test error: 0.89900
[  11]  Training error: 0.90248 Test error: 0.90260
[  12]  Training error: 0.88763 Test error: 0.88650
[  13]  Trainin

# ReLU

In [62]:
batch_size=100;

train_data, train_labels, valid_data, valid_labels=prepare_for_backprop(batch_size, Xtr, Ltr, X_test, L_test)

mlp = MultiLayerPerceptron(layer_config=[784, 100, 100, 10], batch_size=batch_size, activationOut=f_relu)

mlp.evaluate(train_data, train_labels, valid_data, valid_labels,
             eval_train=True, eta=0.05)

print("Done:)\n")

Creating data...
Batch size 100, the number of examples 60000.
Batch size 100, the number of examples 10000.
Done!
Initializing input layer with size 784.
Initializing hidden layer with size 100.
Initializing hidden layer with size 100.
Initializing output layer with size 10.
Done!
Training for 70 epochs...
[   0]  Training error: 0.90085 Test error: 0.89910
[   1]  Training error: 0.88763 Test error: 0.88650
[   2]  Training error: 0.90248 Test error: 0.90260
[   3]  Training error: 0.90248 Test error: 0.90260
[   4]  Training error: 0.90085 Test error: 0.89910
[   5]  Training error: 0.89782 Test error: 0.89900
[   6]  Training error: 0.90128 Test error: 0.90200
[   7]  Training error: 0.89782 Test error: 0.89900
[   8]  Training error: 0.90263 Test error: 0.90180
[   9]  Training error: 0.89558 Test error: 0.89720
[  10]  Training error: 0.89558 Test error: 0.89720
[  11]  Training error: 0.90248 Test error: 0.90260
[  12]  Training error: 0.90137 Test error: 0.90420
[  13]  Trainin

In [41]:
np.maximum(4, np.array([1, 2, 3]))

array([4, 4, 4])